# Balanced Search Trees

**Balanced search trees** are an implementation of symbol tables (with comparable keys) that guarantee efficient operations of search, insert, delete, max, min, rank, floor, ceiling, and select.

## 2-3 Search Trees

Recall from the previous section on Elementary Symbol Tables that the goal for symbol table implementations was $\lg N$ for all operations. **2-3 Trees**, which are left-leaning red-black BSTs, are an old implementation to do this. They allow 1 or 2 keys per node, so there's a **2-node** (one key, two children) or a **3-node** (two keys, three children). The 2-node has two links - one to keys less than the node key and one for keys greater. The 3-node has three links - one for keys less than the smaller key, one for keys between the two keys, and one for keys greater than the larger key.

2-3 trees also have **perfect balance**, so every path from the root to a null link has the same length. They also have **symmetric order** so an in-order traversal (follow left-most paths to keys)  yields the keys in ascending order.

To **insert**, you first search for the key. The easy case is if you end at a 2-node at the bottom, then you just replace that 2-node with a 3-node containing the new inserted key with what was in that 2-node, and add a null link for the third child. To insert a new key to a 3-node at the bottom, first create a temporary 4-node, then move the middle key in the 4-node into the parent. The parent becomes a 3-node, and the 2-node child is split so the children are re-linked (the smaller key becomes the new middle link of the parent and the larger key becomes the right link). If the parent were already a 3-node, it would become a temporary 4-node and that process would propagate up the tree. The only time the height of a 2-3 tree grows is when the root was a 3-node and the process reaches it, so the root has to split.

Splitting a 4-node is a **local** transformation - there are a constant number of operations and they don't touch the subtrees, no matter how many keys are below where the split happens. Each transformation maintains symmetric order and perfect balance.

**Tree height** worst case is $\lg N$ (with all 2-nodes), or best case $\log_{3} N \approx 0.631 \lg N$ (with all 3-nodes). This guarantees **logarithmic** performance for search and insert.

**Implementation** is complicated (see the red-black BST option below instead):
- Maintaining multiple node types is cumbersome
- Need multiple compares to move down the tree
- Need to move back up the tree to split 4-nodes
- Large number of cases for splitting

**Example of inserting in a 2-3 search tree:**

![Wikipedia 2-3 search tree example](https://upload.wikimedia.org/wikipedia/commons/thumb/4/44/2-3_insertion.svg/581px-2-3_insertion.svg.png)

Source: Wikipedia

## Red-Black BSTs

Red-black BSTs are simple data structures that help implement 2-3 trees with very little extra code beyond the basic binary search tree. The idea is to represent a 2-3 tree as a binary search tree, and use "internal" left-leaning links as "glue" for 3-nodes. So the larger of the two keys in a 3-node will be the root in this subtree - its right link goes to keys larger than it and its left link (colored red) connects to the smaller of the two original keys. That key is now a 2-node with a left link to keys smaller and right link to keys that were between the original two keys.

**Black links** connect 2-nodes and 3-nodes, **red links** "glue" nodes within a 3-node.

Some characteristics:
- No node has two red links connected to it
- Every path from the root to a null link has the same number of black links (**perfect black balance**)
- Red links lean left

There's a 1-1 correspondence of left-leaning red-black (LLRB) BSTs and 2-3 trees (think of the red links as horizontal ones, and it looks like a 2-3 tree. You can use the same search code from elementary BST, just ignore the color. It actually runs faster because of better balance. Most other operations (ceiling, selection) are also identical.

Because each node is pointed to by precisely one link (its parent), you can encode the color of the links as data in the node (e.g. `node.left.color == 'RED'` or `node.right.color == 'BLACK'`) and null links are black.

In [1]:
# Node object from BST notes, ignores non-critical methods
class Node:
    def __init__(self, key, val, left=None, right=None, count=1, color='BLACK'):
        self.key = key
        self.val = val
        self.left = left
        self.right = right
        self.count = count  # Number of nodes in subtree including self
        self.color = color  # NEW FOR LLRB BST - color of parent link

    # NEW FOR LLRB BST
    def is_red(self, node_x):
        if node_x is None:
            return False
        return node_x.color == 'RED'
    
    def is_black(self, node_x):
        if node_x is None:
            return True  # Null links are black
        return node_x.color == 'BLACK'

In [2]:
# Test new node functionality
n = Node(2, 2)
print('n color: {}'.format(n.color))
n.right = Node(4, 4)
print('n.right color: {}'.format(n.right.color))
n.left = Node(1, 1, color='RED')
print('n.left.color: {}'.format(n.left.color))
print('Node left link is red: {}'.format(n.is_red(n.left)))
print('Node right link is red: {}'.format(n.is_red(n.right)))
print('Node left link is black: {}'.format(n.is_black(n.left)))
print('Node right link is black: {}'.format(n.is_black(n.right)))

n color: BLACK
n.right color: BLACK
n.left.color: RED
Node left link is red: True
Node right link is red: False
Node left link is black: False
Node right link is black: True


### Red-Black BST Operations

A new operation for red-black BSTs is a **rotation**. During an insertion operation, sometimes you end up with a right-leaning red link (the wrong direction). A rotation will re-orient the link so it leans to the left. Rotations maintain symmetric order and perfect black balance.

**Left Rotation Java Implementation:**

![Example of a right-to-left rotation. Source: Princeton.edu](https://algs4.cs.princeton.edu/33balanced/images/redblack-left-rotate.png)

Sometimes during an insertion, you'll need to temporarily rotate links to have a red right-leaning link before rotating it left. The rotation implementation is similar (see LLRB_BST class for implementation below).

**Right Rotation Java Implementation:**

![Example of a left-to-right rotation. Source: Princeton.edu](https://algs4.cs.princeton.edu/33balanced/images/redblack-right-rotate.png)

Another operation is called a **color flip**, which you use to re-color the local links to split a temporary 4-node. You don't need to change any links, but the parent key will have two red links (both left and right), and will be black itself. You flip the colors so the parent is red and both its links are black.

**Color Flip Java Implementation:**

![Example of a red-black BST color flip. Source: princeton.edu](https://algs4.cs.princeton.edu/33balanced/images/color-flip.png)

**Insertions** use these three operations (left rotation, right rotation, and color flip) to maintain a legal red-black BST with 1-1 correspondence to a 2-3 tree. Here are the main scenarios:

1. Insert into a tree with exactly 1 node
    - **Left:** search ends at the left null link, create a red link to the new node (converts a 2-node into a 3-node)
    - **Right:** search ends the right null link, you attach a new node with a red link on the right, then rotate left to make a legal 3-node
    - **Generalization:** (insert into a 2-node at the bottom) you do a standard BST insert and color the new link red. If it's on the right, rotate left

2. Insert into a tree with 2 nodes (see image). The generalization is to insert into a 3-node at the bottom
    - Do standard BST insert, color the new link red
    - Rotate to balance the 4-node (if needed)
    - Flip colors to pass the red link up one level
    - Rotate to make left-leaning (if needed)

![Example inserting into a tree with 2 nodes](https://x-wei.github.io/images/algoI_week5_1/pasted_image026.png)

The same code handles all cases:
- Right child red, left child black -> rotate left
- Left child, left-left grandchild red -> rotate right
- Both children red -> flip colors

In [3]:
# Full Left-leaning Red-Black Binary Search Tree class
class LL_Red_Black_BST:
    def __init__(self):
        self.root = None

    def size(self):
        return self._size(self.root)

    def _size(self, node_x):
        if node_x is None:
            return 0
        else:
            return node_x.count

    def put(self, key, val):
        # Insert a new key-value node in the BST
        self.root = self._put(self.root, key, val)

    # Insertion code - NEW FOR LLRB BST
    def _put(self, node_h, key, val):
        if (node_h is None):
            return Node(key, val, color='RED')
        if key < node_h.key:
            node_h.left = self._put(node_h.left, key, val)
        elif key > node_h.key:
            node_h.right = self._put(node_h.right, key, val)
        else:
            node_h.val = val

        if (node_h.is_red(node_h.right) and node_h.is_black(node_h.left)):
            # print('Rotating LEFT - parent: {}'.format(node_h.val))
            node_h = self.rotate_left(node_h)
        if (node_h.is_red(node_h.left) and node_h.is_red(node_h.left.left)):
            # print('Rotating RIGHT - parent: {}'.format(node_h.val))
            node_h = self.rotate_right(node_h)
        if (node_h.is_red(node_h.left) and node_h.is_red(node_h.right)):
            # print('FLIP COLORS - parent: {}, left: {}, right: {}'.format(node_h.val, node_h.left.val, node_h.right.val))
            self.flip_colors(node_h)
        
        node_h.count = 1 + self._size(node_h.left) + self._size(node_h.right)
        return node_h

    # Rotate left - NEW FOR LLRB BST
    def rotate_left(self, node_h):
        # node_h is parent with right-leaning red link to node_x
        # Rotates the cluster so node_x is new parent with red link to node_h
        node_x = node_h.right
        node_h.right = node_x.left  # Move middle keys over so they're h's right link
        node_x.left = node_h
        node_x.color = node_h.color  # Assign h's original color to x
        node_h.color = 'RED'
        node_x.count = node_h.count  # Move h's size to x, its new parent
        node_h.count = 1 + self._size(node_h.left) + self._size(node_h.right)
        return node_x

    # Rotate right - NEW FOR LLRB BST
    def rotate_right(self, node_h):
        # node_h is parent with two left red links in a row (child, grandchild)
        # Rotates to the right
        node_x = node_h.left
        node_h.left = node_x.right  # Move middle keys over so they're h's left link
        node_x.right = node_h
        node_x.color = node_h.color  # Assign h's original color to x
        node_h.color = 'RED'
        node_x.count = node_h.count  # Move h's size to x, its new parent
        node_h.count = 1 + self._size(node_h.left) + self._size(node_h.right)
        return node_x

    # Color flip - NEW FOR LLRB BST
    def flip_colors(self, node_h):
        # Flip color of parent and two child node links
        node_h.color == 'RED'
        node_h.left.color = 'BLACK'
        node_h.right.color = 'BLACK'

    def get(self, key):
        # Returns the NODE associated with given key if in tree, None otherwise
        node_x = self.root
        while node_x is not None:
            if key < node_x.key:
                node_x = node_x.left
            elif key > node_x.key:
                node_x = node_x.right
            else:
                return node_x
        return None

    def delete(self, key):
        # Remove the node for a given key
        self.root = self._delete(self.root, key)

    def _delete(self, node_x, key):
        if node_x is None:
            return None
        if key < node_x.key:
            node_x.left = self._delete(node_x.left, key)
        elif key > node_x.key:
            node_x.right = self._delete(node_x.right, key)
        else:
            if node_x.right is None:
                # No right child
                return node_x.left
            if node_x.left is None:
                # No left child
                return node_x.right
            
            # Node has two children - replace with successor
            node_t = node_x
            node_x = self._min(node_t.right)
            node_x.right = self._delete_min(node_t.right)
            node_x.left = node_t.left
        
        # Update subtree counts
        node_x.count = 1 + self._size(node_x.left) + self._size(node_x.right)
        return node_x

    def delete_min(self):
        if self.root:
            self.root = self._delete_min(self.root)

    def _delete_min(self, node_x):
        if node_x.left is None:
            return node_x.right
        node_x.left = self._delete_min(node_x.left)
        node_x.count = 1 + self._size(node_x.left) + self._size(node_x.right)
        return node_x

    def floor(self, key):
        # Return the largest key in the BST <= a given key
        node_x = self._floor(self.root, key)
        if node_x is None:
            return None
        return node_x.key
    
    def _floor(self, node_x, key):
        if node_x is None:
            return None
        if key == node_x.key:
            return node_x
        elif key < node_x.key:
            # Floor is in left subtree
            return self._floor(node_x.left, key)
        
        # Floor may be in right subtree or is root of that subtree
        node_t = self._floor(node_x.right, key)
        if node_t is not None:
            return node_t
        else:
            return node_x
    
    def ceiling(self, key):
        # Return the smallest key in the BST >= a given key
        node_x = self._ceiling(self.root, key)
        if node_x is None:
            return None
        return node_x.key
    
    def _ceiling(self, node_x, key):
        if node_x is None:
            return None
        if key == node_x.key:
            return node_x
        elif key > node_x.key:
            # Ceiling is in right subtree
            return self._ceiling(node_x.right, key)
        
        # Ceiling may be in the left subtree or is root of that subtree
        node_t = self._ceiling(node_x.left, key)
        if node_t is not None:
            return node_t
        else:
            return node_x
    
    def get_max(self):
        # Returns the largest key in the tree
        node_x = self.root
        while node_x is not None:
            if node_x.right is not None:
                node_x = node_x.right
            else:
                return node_x.val
        return None
    
    def get_min(self):
        # Returns the smallest key in the tree
        node_x = self.root
        while node_x is not None:
            if node_x.left is not None:
                node_x = node_x.left
            else:
                return node_x.key
        return None
    
    def _min(self, node_x):
        # Return the node with the smallest key in node_x's subtree
        while node_x is not None:
            if node_x.left is not None:
                node_x = node_x.left
            else:
                return node_x
        return None
    
    def rank(self, key):
        # Returns how many keys in the BST are < given key
        return self._rank(self.root, key)
    
    def _rank(self, node_x, key):
        if node_x is None:
            return 0
        if key < node_x.key:
            return self._rank(node_x.left, key)
        elif key > node_x.key:
            return 1 + self._size(node_x.left) + self._rank(node_x.right, key)
        else:
            return self._size(node_x.left)
             
    def __len__(self):
        return self.size()
    
    def __setitem__(self, key, val):
        self.put(key, val)
    
    def __getitem__(self, key):
        return self.get(key).val

    def __contains__(self, key):
        if self.get(key):
            return True
        else:
            return False

In [4]:
# Test LLRB BST functionality
llrb = LL_Red_Black_BST()

# Add nodes, value equals the key. Check put operation
print('Add nodes to the BST and get values: check put and get operations')
keys = [5, 3, 8, 4, 7, 6, 10, 0]
for k in keys:
    llrb.put(k, k)

# Check get operation
for k in keys:
    temp_node = llrb.get(k)
    print('Key: {}, Val: {}, Link to parent: {}'.format(k, temp_node.val, temp_node.color))

print('\nTree size: {}'.format(llrb.size()))

print('\nCheck ceiling and floor operations')
for k in [7, 2, 11]:
    print('Key: {}\nCeiling: {}\nFloor: {}\n'.format(k, llrb.ceiling(k), llrb.floor(k)))

print('Check max and min operations')
print('Max value: {}'.format(llrb.get_max()))
print('Min value: {}'.format(llrb.get_min()))

print('\nCheck rank operation (number of keys in the tree < given key)')
print('Rank of 6: {}'.format(llrb.rank(6)))
print('Rank of 11: {}'.format(llrb.rank(11)))
print('Rank of 0: {}'.format(llrb.rank(0)))

print('\nCheck dunder methods')
new = 12
llrb[new] = new  # set item
print('Add {0} to table via llrb[{0}] = {0}. New max value is {1}'.format(new, llrb.get_max()))
print('New tree size: {}'.format(len(llrb)))  # len
check = 10
print('Check getitem via llrb[{}] is: {}'.format(check, llrb[check]))  # get item
print('Is 8 in the table? {}'.format(8 in llrb))  # contains
print('Is 9 in the table? {}'.format(9 in llrb))  # contains

print('\nCheck delete operation')
llrb.delete(0)
print('Remove node 0. Tree size: {}. New min value: {}'.format(len(llrb), llrb.get_min()))
llrb.delete(12)
print('Remove node 12. Tree size: {}. New max value: {}'.format(len(llrb), llrb.get_max()))
llrb.delete(8)
print('Remove node 8. Tree size: {}'.format(len(llrb)))

Add nodes to the BST and get values: check put and get operations
Key: 5, Val: 5, Link to parent: RED
Key: 3, Val: 3, Link to parent: BLACK
Key: 8, Val: 8, Link to parent: RED
Key: 4, Val: 4, Link to parent: BLACK
Key: 7, Val: 7, Link to parent: BLACK
Key: 6, Val: 6, Link to parent: BLACK
Key: 10, Val: 10, Link to parent: BLACK
Key: 0, Val: 0, Link to parent: BLACK

Tree size: 8

Check ceiling and floor operations
Key: 7
Ceiling: 7
Floor: 7

Key: 2
Ceiling: 3
Floor: 0

Key: 11
Ceiling: None
Floor: 10

Check max and min operations
Max value: 10
Min value: 0

Check rank operation (number of keys in the tree < given key)
Rank of 6: 4
Rank of 11: 8
Rank of 0: 0

Check dunder methods
Add 12 to table via llrb[12] = 12. New max value is 12
New tree size: 9
Check getitem via llrb[10] is: 10
Is 8 in the table? True
Is 9 in the table? False

Check delete operation
Remove node 0. Tree size: 8. New min value: 3
Remove node 12. Tree size: 7. New max value: 10
Remove node 8. Tree size: 6


## Summary

The worst case (WC) is after $N$ inserts, and the average case (AC) is after $N$ random inserts.

The height of any red-black BST on $n$ keys (regardless of the order of insertion) is guaranteed to be between $\log⁡_{2} n$ and $2 \log_{⁡2}n$.

| Implementation | WC Search | WC Insert | WC Delete | AC Search | AC Insert | AC Delete | Ordered Iteration? |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Sequential Search (unordered list) | $N$ | $N$ | $N$ | $N/2$ | $N$ | $N/2$ | No |
| Binary Search (ordered array) | $\lg N$ | $N$ | $N$ | $\lg N$ | $N/2$ | $N/2$ | Yes |
| Binary Search Tree (BST) | $N$ | $N$ | $N$ | $1.39 \lg N$ | $1.39 \lg N$ | ? | Yes |
| 2-3 Tree | $c \lg N$ | $c \lg N$ | $c \lg N$ | $c \lg N$ | $c \lg N$ | $c \lg N$ | Yes |
| Red-Black BST | $2 \lg N$ | $2 \lg N$ | $2 \lg N$ | $1.00 \lg N$\* | $1.00 \lg N$\* | $1.00 \lg N$ | Yes |

\* Exact coefficient unknown but extremely close to 1.


## B-Trees

**B-trees** are a general version of the red-black BST. One practical application is accessing contiguous blocks of data (a "page", like a file or 4096-byte chunk of data) with a probe (from a disk to memory). The time required to first access the page is much larger than the time to access data within a page. The cost is the number of probes, and the goal is to access the data with a minimum number of probes.

The B-tree generalizes a 2-3 tree by allowing up to $M - 1$ key-link pairs per node. You choose $M$ as large as possible so that $M$ links fit in a page (for example, $M$ is 1024).

- At least 2 key-link pairs at the root (root is a 2-node)
- At least $M / 2$ key-link pairs in other nodes
- External nodes contain client keys
- Internal nodes contain copies of keys to guide search

B-trees are widely used as system symbol tables or for file systems and databases.

### Searching a B-Tree

To search a B-tree, start at the root, then find the interval for the search key and take the corresponding link, and the search terminates in an external node.

### Inserting into a B-Tree

To insert into a B-tree, first search for the new key, then insert at the bottom, finally split nodes with $M$ key-link pairs on the way up the tree.

### Balance in a B-Tree

The proposition is that a search or insertion in a B-tree of order $M$ with $N$ keys requires between $\log_{M-1}N$ and $\log_{M/2}N$ probes. The proof is that all internal nodes have between $M/2$ and $M - 1$ links. In practice, the number of probes is at most 4: $M = 1024$, $N = 62 \text{ billion}$, so $\log_{M/2}N \leq 4$. The optimization is to always keep the root page in memory.

## Geometric Applications of BSTs

One application of symbol trees and the binary search tree data structure is processing geometric data. It uses geometric objects instead of simple keys like integers or strings.

### 1D Range Search

There are many applications for finding how many points in a rectangle ("2D orthogonal range search") or how many rectangles intersect ("orthogonal rectangle intersection"), such as CAD, games, movies, virtual reality, or databases.

**1D Range Search** is an extension of the ordered symbol table. The necessary operations are to *insert* a key-value pair, *search* for key $k$, *delete* key $k$, a *range search* to find all keys between $k_1$ and $k_2$, and a *range count* to get the number of keys between $k_1$ and $k_2$. A common application is database queries.

The geometric interpretation is that keys are points on a line, and you find or count the points in a given 1d interval.

**Implementations:**
- Unordered array: fast insert, slow range search (need to go through all the keys to see whether they're in the range)
- Ordered array: slow insert, binary search for $k_1$ and $k_2$ to do range search

| Data Structure | Insert | Range Count | Range Search |
| --- | --- | --- | --- |
| Unordered Array | 1 | $N$ | $N$ |
| Ordered Array | $N$ | $\log N$ | $R + \log N$ |
| Goal | $\log N$ | $\log N$ | $R + \log N$ |
*$N$ is number of keys and $R$ is number of keys that match*

**The BST implementation for 1D range count:**

```py
def size(lo, hi):
    # Check if the tree contains hi to include that key in count
    if (contains(hi)):
        return rank(hi) - rank(lo) + 1
    else:
        return rank(hi) - rank(lo)
```

This running time is proportional to $\log N$, proof is that the nodes examined are equal to the search path to `lo` plus the search path to `hi`.

**The BST implementation for 1D range search:**

It's a recursive search where you first find all keys in the left subtree (if any fall in the range), check the current node, then recursively find all the keys in the right subtree (if any fall in the range). The running time is proportional to $R + \log N$.

### Line Segment Intersection

**Orthogonal line segment intersection search** is when you find all intersections given $N$ horizontal and vertical line segments.

The naive brute force algorithm checks all pairs of line segments for intersection and operates in quadratic time.

One assumption for the code is the *nondegeneracy assumption* - all $x$ and $y$ coordinates are distinct.

The **sweep-line algorithm** basically sweeps a vertical line from left to right:

- $x$-coordinates define events
- $h$-segment (hit the left endpoint of a horizontal segment): insert $y$-coordinate into BST
- $h$-segment (hit the right endpoint of a horizontal segment): remove $y$-coordinate from BST (that horizontal segment has been processed and no intersections found)
- hit a $v$-segment: do a range search for the interval of $y$'s endpoints (any horizontal line segments $x$ coordinate will be in the BST, so any intersections will return in the range search)

The sweep-line algorithm takes time proportional to $N \log N + R$ to find all $R$ intersections among $N$ orthogonal line segments. The algorithm reduces 2D orthogonal line segment intersection search to 1D range search.

### KD-Trees

How to efficiently process sets of points in space - need 2D keys.

**2D orthogonal range search** uses an extension of the ordered symbol table to 2D keys. It has the same functionality with insert, delete, and search for 2D keys, as well as range search (find all keys that lie in a 2D range) and range count (the number of those keys).

The geometric interpretation is that all keys are points in a plane and you want to find or count the number of points in a given $h$-$v$ rectangle. Some common applications are in networking, circuit design, and databases.

**Grid implementation:**

- Divide space into $M$-by-$M$ grid of squares
- Create list of points contained in each square
- Use 2D array to directly index relevant square
- Insert: add $(x, y)$ to list for corresponding square
- Range search: examine only squares that intersect 2D range query

There's a space-time tradeoff - space is $M^2 + N$ and time is $1 + N/M^2$ per square examined, on average. Want to choose a square size to balance performance (too small wastes space, too large too many points per square). Rule of thumb is $\sqrt N$-by-$\sqrt N$ grid. This is a fast, simple solution for evenly-distributed points.

Points are usually NOT evenly distributed in geometric applications, they're usually **clustered**. The average list is short, but some are long.

**2D Trees**

One adaptation to address geometric data clustering is to use **space-partitioning trees** that represent a recursive subdivision of 2D space. The grid divides space uniformly into squares, the **2D tree** recursively divides space into two half-planes, a **quadtree** recursively divides space into four quadrants, and a **BSP tree** recursively divides space into two regions.

With a 2D tree, first point divides the plane vertically. The next two points are nodes under the first and divide the plane horizontally. Every level in the tree switches horizontal and vertical divisions. The data structure is like a BST, but it alternates using the $x$- and $y$-coordinates as keys. The search function gives the rectangle containing the point and the insert function further subdivides the plane.

To find all points in a query axis-aligned rectangle:

- Check if point in the node lies in the rectangle
- Recursively search left/bottom trees (if any could fall in rectangle)
- Recursively search right/top (if any could fall in rectangle)
- If the splitting line hits the rectangle, you have to check both branches

Another function is to find the nearest neighbor:

- Check distance from point in node to query point
- Recursively search left/bottom trees (if contains closer point)
- Recursively search right/top (if contains closer point)
- Organize method so that it begins by searching for query point

A typical case is $\log N$ but the worst case (even if the tree is balanced) is $N$ (all points in a circle and query point is in the center).

A fun phenomenon in nature to model is how birds (or a school of fish or swarm of insects) behave in a mass group. The **flocking boids** model does this:

- Collision avoidance: point away from *k nearest* boids
- Flock centering: point towards the center of mass of *k nearest* boids
- Velocity matching: update velocity to the average of *k nearest* boids

Shows that 2D trees can efficiently process a lot of information. Also, they can expand to more dimensions, just recursively partition $k$-dimensional space into 2 half-spaces. This is a $k$-dimensional model, or a **KD tree**.

**KD tree** recursively partitions $k$-dimensional space into 2 half-spaces. The implementation is to still use a BST, but cycle through dimensions (like the 2D tree switched between horizontal and vertical levels) - the left subtree has points whose $i^{th}$ coordinate is less than the node point and the right subtree has points whose $i^{th}$ coordinate is greater than the node point.

One similar application is the $N$-body simulation in physics problems (how do a large number of particles behave when affected by same gravitational forces).